# Hiking Trails Data Analysis
This notebook will explore the following:
- Average number of reviews with and without comments per trail
- Distribution of trail difficulty
- Distribution of trail length
- Frequency of terms used in trail reviews 